In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!if not exist "data" mkdir data
!kaggle competitions download -c predict-electricity-consumption
!tar -xf predict-electricity-consumption.zip
!move train.csv data
!move test.csv data
!move sample_submission.csv data
!del predict-electricity-consumption.zip


  0%|          | 0.00/174k [00:00<?, ?B/s]
100%|██████████| 174k/174k [00:00<00:00, 1.11MB/s]
100%|██████████| 174k/174k [00:00<00:00, 1.10MB/s]


        1 file(s) moved.
        1 file(s) moved.
        1 file(s) moved.


In [2]:
train = pd.read_csv('data/train.csv')
train.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
0,2021-09-01 00:00:00+03:00,11.2,10.3,94.0,NaN,NaN,320.0,7.2,16.7,1012.6,2.0,0.09016,0.577
1,2021-09-01 01:00:00+03:00,10.7,9.6,93.0,NaN,NaN,320.0,7.2,13.0,1012.6,2.0,0.09251,0.594
2,2021-09-01 02:00:00+03:00,9.9,9.0,94.0,NaN,NaN,320.0,7.2,13.0,1012.2,2.0,0.08890,0.685
3,2021-09-01 03:00:00+03:00,10.0,8.4,90.0,NaN,NaN,330.0,7.2,13.0,1011.9,1.0,0.08735,1.016
4,2021-09-01 04:00:00+03:00,9.0,8.1,94.0,NaN,NaN,300.0,3.6,13.0,1011.4,2.0,0.08688,0.677


In [3]:
test = pd.read_csv('data/test.csv')
test.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price
0,2022-08-25 00:00:00+03:00,21.1,15.2,69.0,0.0,NaN,340.0,9.0,9.3,1022.0,2.0,0.25533
1,2022-08-25 01:00:00+03:00,20.1,15.1,73.0,0.0,NaN,30.0,6.0,14.8,1022.0,2.0,0.19492
2,2022-08-25 02:00:00+03:00,20.1,15.1,73.0,0.0,NaN,320.0,7.0,13.0,1022.0,2.0,0.18853
3,2022-08-25 03:00:00+03:00,18.7,17.0,90.0,0.0,NaN,0.0,4.0,11.1,1022.4,4.0,0.19947
4,2022-08-25 04:00:00+03:00,18.1,17.1,94.0,0.0,NaN,280.0,7.0,11.1,1022.0,3.0,0.21192


In [4]:
# Count missing values
train.isna().sum()

time              0
temp              0
dwpt              0
rhum              0
prcp           6433
snow           8473
wdir              0
wspd              0
wpgt              0
pres              0
coco            196
el_price          0
consumption       2
dtype: int64

In [5]:
def set_time_index(df):
  # [1444, 4972] - iloc, where time is changing
  df['time'] = pd.to_datetime(df['time'], utc=True).dt.tz_convert('Europe/Tallinn')
  df.set_index('time', drop=True, inplace=True)
  return df

In [6]:
train = set_time_index(train)
train.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
time,,,,,,,,,,,,
2021-09-01 00:00:00+03:00,11.2,10.3,94.0,NaN,NaN,320.0,7.2,16.7,1012.6,2.0,0.09016,0.577
2021-09-01 01:00:00+03:00,10.7,9.6,93.0,NaN,NaN,320.0,7.2,13.0,1012.6,2.0,0.09251,0.594
2021-09-01 02:00:00+03:00,9.9,9.0,94.0,NaN,NaN,320.0,7.2,13.0,1012.2,2.0,0.08890,0.685
2021-09-01 03:00:00+03:00,10.0,8.4,90.0,NaN,NaN,330.0,7.2,13.0,1011.9,1.0,0.08735,1.016
2021-09-01 04:00:00+03:00,9.0,8.1,94.0,NaN,NaN,300.0,3.6,13.0,1011.4,2.0,0.08688,0.677


In [7]:
test = set_time_index(test)
test.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price
time,,,,,,,,,,,
2022-08-25 00:00:00+03:00,21.1,15.2,69.0,0.0,NaN,340.0,9.0,9.3,1022.0,2.0,0.25533
2022-08-25 01:00:00+03:00,20.1,15.1,73.0,0.0,NaN,30.0,6.0,14.8,1022.0,2.0,0.19492
2022-08-25 02:00:00+03:00,20.1,15.1,73.0,0.0,NaN,320.0,7.0,13.0,1022.0,2.0,0.18853
2022-08-25 03:00:00+03:00,18.7,17.0,90.0,0.0,NaN,0.0,4.0,11.1,1022.4,4.0,0.19947
2022-08-25 04:00:00+03:00,18.1,17.1,94.0,0.0,NaN,280.0,7.0,11.1,1022.0,3.0,0.21192


In [8]:
def get_corr_consumption(df):
  df_corr = df.corr(method="pearson")
  print(df_corr.shape)
  print("Correlation with consumption:")
  return pd.DataFrame(df_corr["consumption"].sort_values(ascending=False))

In [9]:
df_corr_train_consumption = get_corr_consumption(train)
df_corr_train_consumption

(12, 12)
Correlation with consumption:


,consumption
consumption,1.000000
coco,0.133539
rhum,0.094408
wpgt,0.072206
wspd,0.048840
snow,0.004163
wdir,-0.005009
prcp,-0.010668
pres,-0.068940
el_price,-0.124740


In [10]:
# train_lags = train.copy()
# columns_for_lags = ['temp', 'dwpt', 'el_price']
# lags = [1, 24, 48, 72]
# for col in columns_for_lags:
#   for lag in lags:
#     train_lags[f'{col}_lag_{lag}'] = train_lags[col].shift(lag)
# train_lags.drop(index=train_lags.index[:max(lags)], axis=1, inplace=True)
# train_lags.head()

In [11]:
from darts import TimeSeries

In [12]:
# Remove time zone
train.index = train.index.tz_localize(None)

In [13]:
iloc_start_time = 5000

In [14]:
ts_C = TimeSeries.from_series(train.iloc[iloc_start_time:]['consumption'])
ts_C

<TimeSeries (DataArray) (time: 3592, component: 1, sample: 1)>
array([[[0.357]],

       [[1.31 ]],

       [[0.362]],

       ...,

       [[0.5  ]],

       [[2.321]],

       [[0.678]]])
Coordinates:
  * time       (time) datetime64[ns] 2022-03-28T08:00:00 ... 2022-08-24T23:00:00
  * component  (component) object 'consumption'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [15]:
ts_feat = TimeSeries.from_series(train.iloc[iloc_start_time:].drop('consumption', axis=1))
ts_feat

<TimeSeries (DataArray) (time: 3592, component: 11, sample: 1)>
array([[[3.4000e+00],
        [1.1000e+00],
        [8.5000e+01],
        ...,
        [9.9780e+02],
        [1.8000e+01],
        [1.1845e-01]],

       [[4.5000e+00],
        [1.0000e+00],
        [7.8000e+01],
        ...,
        [9.9700e+02],
        [4.0000e+00],
        [1.1844e-01]],

       [[5.4000e+00],
        [1.0000e+00],
        [7.3000e+01],
        ...,
...
        ...,
        [1.0209e+03],
        [4.0000e+00],
        [4.3149e-01]],

       [[2.2100e+01],
        [1.7000e+01],
        [7.3000e+01],
        ...,
        [1.0210e+03],
        [2.0000e+00],
        [5.5203e-01]],

       [[2.2100e+01],
        [1.8100e+01],
        [7.8000e+01],
        ...,
        [1.0210e+03],
        [2.0000e+00],
        [4.3108e-01]]])
Coordinates:
  * time       (time) datetime64[ns] 2022-03-28T08:00:00 ... 2022-08-24T23:00:00
  * component  (component) object 'temp' 'dwpt' 'rhum' ... 'coco' 'el_price'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [16]:
print("components (columns) of consumption time series:", ts_C.components)
print("duration:",ts_C.duration)
print("frequency:",ts_C.freq)
print("frequency:",ts_C.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_C.has_datetime_index)
print("deterministic:",ts_C.is_deterministic)
print("univariate:",ts_C.is_univariate)

components (columns) of consumption time series: Index(['consumption'], dtype='object', name='component')
duration: 149 days 15:00:00
frequency: <Hour>
frequency: H
has date time index? (or else, it must have an integer index): True
deterministic: True
univariate: True


In [17]:
print("components (columns) of feature time series:", ts_feat.components)
print("duration:",ts_feat.duration)
print("frequency:",ts_feat.freq)
print("frequency:",ts_feat.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_feat.has_datetime_index)
print("deterministic:",ts_feat.is_deterministic)
print("univariate:",ts_feat.is_univariate)

components (columns) of feature time series: Index(['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'coco', 'el_price'],
      dtype='object', name='component')
duration: 149 days 15:00:00
frequency: <Hour>
frequency: H
has date time index? (or else, it must have an integer index): True
deterministic: True
univariate: False


In [18]:
from darts.dataprocessing.transformers import Scaler

In [19]:
# train/val split
val_size_d = 7 * 24 

ts_C_train = ts_C[:len(ts_C) - val_size_d]
ts_C_val = ts_C[len(ts_C) - val_size_d:]

ts_feat_train = ts_feat[:len(ts_feat) - val_size_d]
ts_feat_val = ts_feat[len(ts_feat) - val_size_d:]

scaler_consumption = Scaler()
scaler_consumption.fit_transform(ts_C_train)
ts_C_train_trf = scaler_consumption.transform(ts_C_train)
ts_C_val_trf = scaler_consumption.transform(ts_C_val)    
ts_C_trf = scaler_consumption.transform(ts_C)

scaler_feat = Scaler()
scaler_feat.fit_transform(ts_feat_train)
ts_feat_train_trf = scaler_feat.transform(ts_feat_train)
ts_feat_val_trf = scaler_feat.transform(ts_feat_val)    
ts_feat_trf = scaler_feat.transform(ts_feat)

print('TimeSeries: consumption')
print("training start:", ts_C_train.start_time())
print("training end:", ts_C_train.end_time())
print("training duration:",ts_C_train.duration)
print("validation start:", ts_C_val.start_time())
print("validation end:", ts_C_val.end_time())
print("validation duration:", ts_C_val.duration)

print('\n')
print('TimeSeries: features')
print("training start:", ts_feat_train.start_time())
print("training end:", ts_feat_train.end_time())
print("training duration:",ts_feat_train.duration)
print("validation start:", ts_feat_val.start_time())
print("validation end:", ts_feat_val.end_time())
print("validation duration:", ts_feat_val.duration)


TimeSeries: consumption
training start: 2022-03-28 08:00:00
training end: 2022-08-17 23:00:00
training duration: 142 days 15:00:00
validation start: 2022-08-18 00:00:00
validation end: 2022-08-24 23:00:00
validation duration: 6 days 23:00:00


TimeSeries: features
training start: 2022-03-28 08:00:00
training end: 2022-08-17 23:00:00
training duration: 142 days 15:00:00
validation start: 2022-08-18 00:00:00
validation end: 2022-08-24 23:00:00
validation duration: 6 days 23:00:00


In [37]:
from darts.models import TransformerModel, TFTModel

In [21]:
LOAD = False         # True = load previously saved model from disk?  False = (re)train the model
SAVE = "\_TForm_model.pth.tar"   # file name to save the model under

EPOCHS = 50
INLEN = 32          # input size
FEAT = 24           # d_model = number of expected features in the inputs, up to 512    
HEADS = 4           # default 8
ENCODE = 4          # encoder layers
DECODE = 4          # decoder layers
DIM_FF = 128        # dimensions of the feedforward network, default 2048
BATCH = 32          # batch size
ACTF = "relu"       # activation function, relu (default) or gelu
SCHLEARN = None     # a PyTorch learning rate scheduler; None = constant rate
LEARN = 1e-3        # learning rate
VALWAIT = 1         # epochs to wait before evaluating the loss on the test/validation set
DROPOUT = 0.1       # dropout rate
N_FC = 1            # output size
QUANTILES = [0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99] # default quantiles for QuantileRegression

RAND = 42           # random seed
N_SAMPLES = 100     # number of times a prediction is sampled from a probabilistic model
N_JOBS = 2          # parallel processors to use;  -1 = all processors

In [22]:
from darts.utils.likelihood_models import QuantileRegression

In [23]:
model = TransformerModel(
                    input_chunk_length = INLEN,
                    output_chunk_length = N_FC,
                    batch_size = BATCH,
                    n_epochs = EPOCHS,
                    model_name = "Transformer_price",
                    nr_epochs_val_period = VALWAIT,
                    d_model = FEAT,
                    nhead = HEADS,
                    num_encoder_layers = ENCODE,
                    num_decoder_layers = DECODE,
                    dim_feedforward = DIM_FF,
                    dropout = DROPOUT,
                    activation = ACTF,
                    random_state=RAND,
                    likelihood=QuantileRegression(quantiles=QUANTILES), 
                    optimizer_kwargs={'lr': LEARN},
                    # add_encoders={"cyclic": {"future": ["hour", "dayofweek", "month"]}},
                    save_checkpoints=True,
                    force_reset=True,
                    pl_trainer_kwargs={
                      "accelerator": "gpu",
                      "devices": [0]
                    }
                    )

In [69]:
# training: load a saved model or (re)train
if LOAD:
    print("have loaded a previously saved model from disk:" + SAVE)
    model = TransformerModel.load_model(SAVE)                            # load previously model from disk 
else:
    model.fit(ts_C_train_trf, 
                past_covariates=ts_feat_train_trf, 
                verbose=True,
                val_series=ts_C_val_trf,
                val_past_covariates=ts_feat_val_trf)
    print("have saved the model after training:", SAVE)
    model.save(SAVE)

2022-12-01 16:22:38 main_logger ERROR: ValueError: Some future_covariates have been provided to a PastCovariates model. These models support only past_covariates.


ValueError: Some future_covariates have been provided to a PastCovariates model. These models support only past_covariates.

In [36]:
ts_C_pred = model.predict(  n=len(ts_feat_val_trf),
                            past_covariates=ts_feat_val_trf,
                            num_samples=1,   
                            n_jobs=N_JOBS, 
                            verbose=True)

2022-12-01 15:31:52 main_logger ERROR: ValueError: For the given forecasting case, the provided past covariates at dataset index `0` do not extend far enough into the past. The past covariates must start at time step `2022-08-16 16:00:00`, whereas now they start at time step `2022-08-18 00:00:00`.


ValueError: For the given forecasting case, the provided past covariates at dataset index `0` do not extend far enough into the past. The past covariates must start at time step `2022-08-16 16:00:00`, whereas now they start at time step `2022-08-18 00:00:00`.

In [39]:
import torch
from darts.models import TFTModel

In [106]:
model2 = TFTModel(
  input_chunk_length = INLEN,
  output_chunk_length = N_FC,
  hidden_size=64,
  lstm_layers=1,
  num_attention_heads=HEADS,
  dropout = DROPOUT,
  batch_size = BATCH,
  n_epochs = EPOCHS,

  likelihood=None,
  # loss_fn=torch.nn.MSELoss()

  nr_epochs_val_period = VALWAIT,
  
  random_state=RAND,

  save_checkpoints=True,
  force_reset=True,
  pl_trainer_kwargs={
    "accelerator": "gpu",
    "devices": -1,
    'auto_select_gpus': True
    }
  
)

In [120]:
model2.fit(
  ts_C_train_trf,
  future_covariates=ts_feat_train_trf,
  val_series=ts_C_val_trf,
  val_future_covariates=ts_feat_full_trf.drop_before(pd.Timestamp('2022-08-16 15:00:00')),
  verbose=True
)

2022-12-01 18:03:36 pytorch_lightning.trainer.connectors.accelerator_connector INFO: Auto select gpus: [0]
2022-12-01 18:03:36 pytorch_lightning.utilities.rank_zero INFO: GPU available: True (cuda), used: True
2022-12-01 18:03:36 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-12-01 18:03:36 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-12-01 18:03:36 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-12-01 18:03:36 darts.models.forecasting.torch_forecasting_model WARNING: Attempting to retrain the model without resuming from a checkpoint. This is currently discouraged. Consider setting `save_checkpoints` to `True` and specifying `model_name` at model creation. Then call `model = TFTModel.load_from_checkpoint(model_name, best=False)`. Finally, train the model with `model.fit(..., epochs=new_epochs)` where `new_epochs` is the sum of (epochs already trained + some additional

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [85]:
from darts import concatenate

In [113]:
ts_feat_full_trf = concatenate([ts_feat_train_trf, ts_feat_val_trf], axis=0)

In [114]:
ts_feat_full_trf.drop_before(pd.Timestamp('2022-08-16 15:00:00')).start_time()

Timestamp('2022-08-16 16:00:00', freq='H')

In [117]:
model2.predict(n=1, future_covariates=ts_feat_full_trf.drop_before(pd.Timestamp('2022-08-16 15:00:00')))

2022-12-01 18:02:22 pytorch_lightning.accelerators.cuda INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 106it [00:00, ?it/s]

<TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
array([[[nan]]])
Coordinates:
  * time       (time) datetime64[ns] 2022-08-18
  * component  (component) object 'consumption'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None